In [1]:
import csv
from random import shuffle

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece
from keras import backend as K
from keras.engine import Layer
import keras.layers as layers
import keras.optimizers as optimizers
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

W0605 20:44:43.210646 140019569018624 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [2]:
csv_path = './datasets.csv'
datasets = {}
with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    for index, row in enumerate(reader):
        # 先頭をlabelとし、それ以降をデータとする
        datasets[row[0]] = row[1:len(row)]

In [3]:
x_train = []
y_train = []
labels = []

In [4]:
for key, sentences in datasets.items():
    labels.append(key)
    label_index = labels.index(key)
    for sentence in sentences:
        x_train.append(sentence)
        y_train.append(label_index)

In [5]:
x_train, y_train, labels

(['帽子買いたい',
  'ジーパン欲しい',
  'Tシャツ買いたい',
  '靴がボロくなった',
  'ジャケット買わないと',
  '夕食の食材買いたい',
  '野菜買わないと',
  '卵足りない',
  '牛乳飲みたい',
  '日用品がなくなってきた',
  '腹減った',
  '飯くいたい',
  'ランチどこにしよう',
  'ステーキが欲しい',
  'お腹ぺこぺこ',
  'コーヒー飲みたい',
  'タバコ買いたい',
  'ひと息つきたい',
  'お菓子買いたい',
  '休憩しよう',
  '漏れそう',
  '用を足したい',
  'お手洗いはどこですか',
  '化粧室はどこ',
  '手を洗いたい',
  '体調悪い',
  'お腹痛い',
  '頭痛がする',
  '人間ドック受けたい',
  '風邪ひいた',
  '車が止まりそう',
  'タイヤがパンクした',
  'ガス欠',
  'ワイパー交換',
  'オイルが古い'],
 [0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  6,
  6,
  6,
  6,
  6],
 ['ファッション', 'スーパー', 'レストラン', 'コンビニ', 'トイレ', '病院', 'ガゾリンスタンド'])

In [6]:
train_data = list(zip(x_train, y_train))
shuffle(train_data)
x_train = [d[0] for d in train_data]
y_train = [d[1] for d in train_data]

In [7]:
x_train, y_train, labels

(['手を洗いたい',
  '体調悪い',
  'ランチどこにしよう',
  '人間ドック受けたい',
  '漏れそう',
  'ひと息つきたい',
  '卵足りない',
  '休憩しよう',
  '用を足したい',
  '牛乳飲みたい',
  'ワイパー交換',
  'お菓子買いたい',
  '化粧室はどこ',
  'ガス欠',
  '飯くいたい',
  '風邪ひいた',
  '夕食の食材買いたい',
  'コーヒー飲みたい',
  '野菜買わないと',
  'お手洗いはどこですか',
  '帽子買いたい',
  'Tシャツ買いたい',
  'オイルが古い',
  'タバコ買いたい',
  '腹減った',
  'ステーキが欲しい',
  'ジーパン欲しい',
  'お腹ぺこぺこ',
  'ジャケット買わないと',
  '靴がボロくなった',
  'タイヤがパンクした',
  '頭痛がする',
  '日用品がなくなってきた',
  'お腹痛い',
  '車が止まりそう'],
 [4,
  5,
  2,
  5,
  4,
  3,
  1,
  3,
  4,
  1,
  6,
  3,
  4,
  6,
  2,
  5,
  1,
  3,
  1,
  4,
  0,
  0,
  6,
  3,
  2,
  2,
  0,
  2,
  0,
  0,
  6,
  5,
  1,
  5,
  6],
 ['ファッション', 'スーパー', 'レストラン', 'コンビニ', 'トイレ', '病院', 'ガゾリンスタンド'])

In [8]:
x_train = np.array(x_train)
y_train = np_utils.to_categorical(np.array(y_train))

In [9]:
x_train, y_train, labels

(array(['手を洗いたい', '体調悪い', 'ランチどこにしよう', '人間ドック受けたい', '漏れそう', 'ひと息つきたい',
        '卵足りない', '休憩しよう', '用を足したい', '牛乳飲みたい', 'ワイパー交換', 'お菓子買いたい',
        '化粧室はどこ', 'ガス欠', '飯くいたい', '風邪ひいた', '夕食の食材買いたい', 'コーヒー飲みたい',
        '野菜買わないと', 'お手洗いはどこですか', '帽子買いたい', 'Tシャツ買いたい', 'オイルが古い', 'タバコ買いたい',
        '腹減った', 'ステーキが欲しい', 'ジーパン欲しい', 'お腹ぺこぺこ', 'ジャケット買わないと', '靴がボロくなった',
        'タイヤがパンクした', '頭痛がする', '日用品がなくなってきた', 'お腹痛い', '車が止まりそう'],
       dtype='<U11'), array([[0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 

In [10]:
model = None
model_path = 'models/usex.h5'
log_path = 'logs/tboard'

In [11]:
# load model
# model = load_model(model_path, custom_objects={'USEXEmbeddingLayer': USEXEmbeddingLayer})

In [12]:
# define EmbeddingLayer
class USEXEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.name = 'USEXEmbeddingLayer'
        self.trainable = kwargs['trainable'] if 'trainable' in kwargs else False
        super(USEXEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.usex = hub.Module(
            'https://tfhub.dev/google/universal-sentence-encoder-xling-many/1',
            trainable=self.trainable,
            name="{}_module".format(self.name),
        )
        super(USEXEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.usex(
            K.squeeze(K.cast(x, K.tf.string), axis=1),
            as_dict=True,
            signature='default',
        )['default']
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 512)

In [13]:
# define model
inputs = layers.Input(shape=(1,), dtype='string')
outputs = USEXEmbeddingLayer()(inputs)
outputs = layers.Dense(512, activation='relu')(outputs)
outputs = layers.BatchNormalization()(outputs)
outputs = layers.Dropout(0.5)(outputs)
outputs = layers.Dense(512, activation='relu')(outputs)
outputs = layers.BatchNormalization()(outputs)
outputs = layers.Dropout(0.5)(outputs)
outputs = layers.Dense(len(labels), activation='softmax')(outputs)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0605 20:45:09.730326 140019569018624 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [14]:
model = Model(inputs=[inputs], outputs=outputs)
model.compile(
    optimizer=optimizers.rmsprop(
        lr=0.001,
        rho=0.9,
        epsilon=None,
        decay=0.0,
    ),
    loss='categorical_crossentropy',
    metrics=['acc'],
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
usex_embedding_layer_1 (USEX (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
__________

In [15]:
# train
model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=1024,
    validation_split=0.1,
    shuffle=True,
    callbacks=[
        EarlyStopping(
            monitor='val_acc',
            patience=1,
        ),
        ModelCheckpoint(
            filepath=model_path,
            monitor='val_loss',
            save_best_only=True,
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=1,
        ),
        TensorBoard(
            log_dir=log_path,
            write_graph=True,
        )
    ],
)

Train on 31 samples, validate on 4 samples
Epoch 1/100
31/31 [==============================] - 4s 126ms/step - loss: 3.5861 - acc: 0.0968 - val_loss: 1.1221 - val_acc: 0.7500
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 0.6136 - acc: 0.8387 - val_loss: 1.1953 - val_acc: 0.5000


In [16]:
# predict
sentence = 'ディナーどこにしよう'

results = model.predict(np.array([sentence]))

In [17]:
results

array([[4.7869139e-04, 1.1557645e-02, 9.2194986e-01, 2.0164052e-02,
        4.4174928e-02, 1.0878982e-03, 5.8688014e-04]], dtype=float32)

In [18]:
result = results[0]
indexes = list(range(len(labels)))
predictions = dict(zip(indexes, result))
predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
predictions = predictions[0:3]
for prediction in predictions:
    label = labels[prediction[0]]
    score = '{:.2%}'.format(prediction[1])
    print('{}:{}'.format(score, label))

92.19%:レストラン
4.42%:トイレ
2.02%:コンビニ


In [19]:
def generate_predicter(model, labels):
    indexes = list(range(len(labels)))

    def predicter(sentences):
        results = model.predict(np.array(sentences))
        for sentence_index, result in enumerate(results):
            sentence = sentences[sentence_index]
            print('====================')
            print('q: {}'.format(sentence))
            predictions = dict(zip(indexes, result))
            predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
            predictions = predictions[0:5]
            for prediction in predictions:
                index = prediction[0]
                label = labels[index]
                score = prediction[1]
                print('\n----------\nscore:{}\n{}'.format('{:.2%}'.format(score), label))

    return predicter

In [20]:
predicter = generate_predicter(model, labels)

In [21]:
predicter(['車が壊れた'])

q: 車が壊れた

----------
score:95.10%
ガゾリンスタンド

----------
score:2.25%
スーパー

----------
score:0.91%
トイレ

----------
score:0.88%
ファッション

----------
score:0.46%
病院


In [22]:
# save pb
serving_model_path = 'models/serving/1'
tf.saved_model.simple_save(
    K.get_session(),
    serving_model_path,
    inputs={'inputs': model.input},
    outputs={t.name: t for t in model.outputs},
)

Instructions for updating:
Pass your op to the equivalent parameter main_op instead.


W0605 20:46:15.939420 140019569018624 tf_logging.py:125] From /usr/local/lib/python3.6/site-packages/tensorflow/python/saved_model/simple_save.py:85: calling SavedModelBuilder.add_meta_graph_and_variables (from tensorflow.python.saved_model.builder_impl) with legacy_init_op is deprecated and will be removed in a future version.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.


INFO:tensorflow:Assets added to graph.


I0605 20:46:15.941284 140019569018624 tf_logging.py:115] Assets added to graph.


INFO:tensorflow:No assets to write.


I0605 20:46:15.943156 140019569018624 tf_logging.py:115] No assets to write.


INFO:tensorflow:SavedModel written to: models/serving/1/saved_model.pb


I0605 20:46:31.195915 140019569018624 tf_logging.py:115] SavedModel written to: models/serving/1/saved_model.pb
